In [1]:
import pickle
import train
import build_features
import numpy as np
import time
import os

In [3]:
#Get the data
d1 = pickle.load(open('../python objects/patientdata_20170823.pkl', 'rb'))
d1mom = pickle.load(open('../python objects/patient_mother_data_20170724.pkl', 'rb'))
lat_lon_dic = pickle.load(open('../python objects/lat_lon_data_20170920.pkl', 'rb'))
env_dic= pickle.load(open('../python objects/census_data_20170920.pkl', 'rb'))
d1mom_hist = pickle.load(open('../python objects/full_lutheran_mother_data.pkl', 'rb'))



In [5]:
prec_total = []
recall_total = []
spec_total = []
titles_total = []
model_list = []
auc_list = []
r2_list = []


# timestr = time.strftime("%Y%m%d")
# newdir='../outputs_age_analyses'+timestr
# os.mkdir(newdir)

agex_low = 4.5
agex_high = 5.5
months_from = 0
for months_to in [6,12,18,24,36,48]:

    #Create the overall data sets
    x1_no_maternal,y1,y1label,feature_headers,mrns = build_features.call_build_function(d1, d1mom, {}, lat_lon_dic, env_dic, agex_low, agex_high, months_from, months_to, False)
    x1_maternal,y1,y1label,feature_headers,mrns = build_features.call_build_function(d1, d1mom, d1mom_hist, lat_lon_dic, env_dic, agex_low, agex_high, months_from, months_to, False)

    # pickle.dump(x1_no_maternal, open('x_no_maternal_'+timestr+'.pkl', 'wb'))
    # pickle.dump(x1_maternal, open('x_maternal_'+timestr+'.pkl', 'wb'))
#     np.save(newdir+'/x_no_maternal_pred_at_'+str(months_to)+'months.npy', x1_no_maternal)
#     np.save(newdir+'/x_w_maternal_pred_at_'+str(months_to)+'months.npy', x1_maternal)
#     np.save(newdir+'/y_pred_at_'+str(months_to)+'months.npy', y1)
#     np.save(newdir+'/y_label_pred_at_'+str(months_to)+'months.npy', y1label)
#     pickle.dump(feature_headers, open(newdir+'/feature_headers_pred_at_'+str(months_to)+'months.pkl', 'wb'))
#     pickle.dump(mrns, open(newdir+'/mrns_pred_at_'+str(months_to)+'months.pkl', 'wb'))

    #Run all the models
    # filter_str = ['Vital: Wt-latest', 'Vital: BMI-latest']
    if months_to == 6:
        filter_str = ['Vital: Wt-avg0to1','Vital: Wt-avg1to3','Vital: BMI-avg0to1','Vital: BMI-avg1to3']
    elif months_to == 12 or months_to == 18:
        filter_str = ['Vital: Wt-avg0to1','Vital: Wt-avg1to3','Vital: Wt-avg10to13','Vital: BMI-avg0to1','Vital: BMI-avg1to3','Vital: BMI-avg10to13']
    else:
        filter_str = ['Vital: Wt-avg0to1','Vital: Wt-avg1to3','Vital: Wt-avg10to13','Vital: Wt-avg19to24','Vital: BMI-avg0to1','Vital: BMI-avg1to3','Vital: BMI-avg10to13','Vital: BMI-avg19to24']

    print('\nPredicting obesity at 5 years from %d months' %(months_to))
    for modeltype_ix in ['lasso','randomforest', 'gradientboost']:
        for ix, gender in enumerate(['boys', 'girls']):
            for mother_hist_ix in ['no_maternal', 'maternal']:
                x1 = x1_no_maternal if mother_hist_ix == 'no_maternal' else x1_maternal
                filterstr_ix = 'Gender:0' if ix == 0 else 'Gender:1'
                gender_clean = 'boys' if ix==0 else 'girls'

                print('\n' + modeltype_ix + '\t' + filterstr_ix + '\t' + mother_hist_ix + '@ ' + str(months_to) + ' months')
                (model, x2, y2, y2label, feature_headers, xtrain, ytrain, ytrainlabel, mrnstrain, xtest, ytest, ytestlabel, mrnstest,
                filterSTR, sig_headers, centroids, hnew, standardDevCentroids, cnt_clusters, muxnew, stdxnew, mrns2,
                prec_list, recall_list, spec_list, test_auc_mean, test_auc_mean_ste, r2test_mean, r2test_ste) = \
                        train.train_regression_model_for_bmi({}, {}, {}, {}, {}, x1, y1, y1label, feature_headers, mrns, agex_low, agex_high, months_from, months_to,
                                filterSTR=filter_str + [filterstr_ix],
                                variablesubset=[],
                                modelType=modeltype_ix,
                                return_data_for_error_analysis=False,
                                return_data=True,
                                return_data_transformed=True,
                                return_train_test_data=True,
                                do_impute=False)

#                 np.save(newdir+'/train_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,str(months_to),'months.npy']), np.hstack((xtrain,mrnstrain.reshape(-1,1),ytrain.reshape(-1,1),ytrainlabel.reshape(-1,1))))
#                 np.save(newdir+'/test_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,str(months_to),'months.npy']), np.hstack((xtest,mrnstest.reshape(-1,1),ytest.reshape(-1,1),ytestlabel.reshape(-1,1))))
#                 np.save(newdir+'/data_transformed_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,str(months_to)+'months.npy']), np.hstack((x2,mrns2.reshape(-1,1),y2.reshape(-1,1),y2label.reshape(-1,1))))
#                 pickle.dump(feature_headers, open(newdir+'/feature_headers_pred_at_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,str(months_to)+'months_transformed.pkl']), 'wb'))

                titles_total.append(gender_clean + ' ' + mother_hist_ix + ' - model: ' + modeltype_ix + '@ ' + str(months_to) + 'months')
                model_list.append(model)
                prec_total.append(prec_list)
                recall_total.append(recall_list)
                spec_total.append(spec_list)
                auc_list.append([test_auc_mean, test_auc_mean_ste])
                r2_list.append([r2test_mean, r2test_ste])

                # without vitals and maternal historical data
                print(modeltype_ix + '\t' + filterstr_ix + '\t' + mother_hist_ix + '@ ' + str(months_to) + ' months w/o vitals and maternal historical data')
                (model, x2, y2, y2label, feature_headers, xtrain, ytrain, ytrainlabel, mrnstrain, xtest, ytest, ytestlabel, mrnstest,
                filterSTR, sig_headers, centroids, hnew, standardDevCentroids, cnt_clusters, muxnew, stdxnew, mrns2,
                prec_list, recall_list, spec_list, test_auc_mean, test_auc_mean_ste, r2test_mean, r2test_ste) = \
                        train.train_regression_model_for_bmi({}, {}, {}, {}, {}, x1, y1, y1label, feature_headers, mrns, agex_low, agex_high, months_from, months_to,
                                filterSTR=[filterstr_ix],
                                variablesubset=['Census','Diagnosis','Ethnicity','Gender','Lab','MatDeliveryAge',
                                    'Medication','Newborn Diagnosis','Prim_Insur','Race','Second_Insur','Zipcode'],
                                modelType=modeltype_ix,
                                return_data_for_error_analysis=False,
                                return_data=True,
                                return_data_transformed=True,
                                return_train_test_data=True,
                                do_impute=False)

#                 np.save(newdir+'/train_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'no_vitals_no_mat',str(months_to),'months.npy']), np.hstack((xtrain,mrnstrain.reshape(-1,1),ytrain.reshape(-1,1),ytrainlabel.reshape(-1,1))))
#                 np.save(newdir+'/test_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'no_vitals_no_mat',str(months_to),'months.npy']), np.hstack((xtest,mrnstest.reshape(-1,1),ytest.reshape(-1,1),ytestlabel.reshape(-1,1))))
#                 np.save(newdir+'/data_transformed_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'no_vitals_no_mat',str(months_to)+'months.npy']), np.hstack((x2,mrns2.reshape(-1,1),y2.reshape(-1,1),y2label.reshape(-1,1))))
#                 pickle.dump(feature_headers, open(newdir+'/feature_headers_pred_at_'+'_'.join([modeltype_ix,'no_vitals_no_mat',gender_clean,mother_hist_ix,str(months_to)+'months_transformed.pkl']), 'wb'))

                titles_total.append(gender_clean + ' ' + mother_hist_ix + ' w/o vitals and maternal' + ' - model: ' + modeltype_ix + '@ ' + str(months_to) + 'months')
                model_list.append(model)
                prec_total.append(prec_list)
                recall_total.append(recall_list)
                spec_total.append(spec_list)
                auc_list.append([test_auc_mean, test_auc_mean_ste])
                r2_list.append([r2test_mean, r2test_ste])

            # 'Vital: BMI-latest' only model
            print(modeltype_ix + '\t' + filterstr_ix + '\t' + ' Vital: BMI-latest @ ' + str(months_to) + ' months')
            (model, x2, y2, y2label, feature_headers, xtrain, ytrain, ytrainlabel, mrnstrain, xtest, ytest, ytestlabel, mrnstest,
            filterSTR, sig_headers, centroids, hnew, standardDevCentroids, cnt_clusters, muxnew, stdxnew, mrns2,
            prec_list, recall_list, spec_list, test_auc_mean, test_auc_mean_ste, r2test_mean, r2test_ste) = \
                    train.train_regression_model_for_bmi({}, {}, {}, {}, {}, x1, y1, y1label, feature_headers, agex_low, agex_high, months_from, months_to,
                            filterSTR=filter_str + [filterstr_ix],
                            variablesubset=['Vital: BMI-latest'],
                            modelType=modeltype_ix,
                            return_data_for_error_analysis=False,
                            return_data=True,
                            return_data_transformed=True,
                            return_train_test_data=True,
                            do_impute=False)

#             np.save(newdir+'/train_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: BMI-latest',str(months_to),'months.npy']), np.hstack((xtrain,mrnstrain.reshape(-1,1),ytrain.reshape(-1,1),ytrainlabel.reshape(-1,1))))
#             np.save(newdir+'/test_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: BMI-latest',str(months_to),'months.npy']), np.hstack((xtest,mrnstest.reshape(-1,1),ytest.reshape(-1,1),ytestlabel.reshape(-1,1))))
#             np.save(newdir+'/data_transformed_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: BMI-latest',str(months_to)+'months.npy']), np.hstack((x2,mrns2.reshape(-1,1),y2.reshape(-1,1),y2label.reshape(-1,1))))
#             pickle.dump(feature_headers, open(newdir+'/feature_headers_pred_at_'+'_'.join([modeltype_ix,'Vital: BMI-latest',gender_clean,mother_hist_ix,str(months_to)+'months_transformed.pkl']), 'wb'))

            titles_total.append(gender_clean + ' - model: ' + modeltype_ix + ' - ' + 'Vital: BMI-latest @ ' + str(months_to) + 'months')
            model_list.append(model)
            prec_total.append(prec_list)
            recall_total.append(recall_list)
            spec_total.append(spec_list)
            auc_ste_list.append([test_auc_mean, test_auc_mean_ste])
            r2_list.append([r2test_mean, r2test_ste])

            # 'Vital: Wt-latest' only model
            print(modeltype_ix + '\t' + filterstr_ix + '\t' + ' Vital: Wt-latest @ ' + str(months_to) + ' months')
            (model, x2, y2, y2label, feature_headers, xtrain, ytrain, ytrainlabel, mrnstrain, xtest, ytest, ytestlabel, mrnstest,
            filterSTR, sig_headers, centroids, hnew, standardDevCentroids, cnt_clusters, muxnew, stdxnew, mrns2,
            prec_list, recall_list, spec_list, test_auc_mean, test_auc_mean_ste, r2test_mean, r2test_ste) = \
                    train.train_regression_model_for_bmi({}, {}, {}, {}, {}, x1, y1, y1label, feature_headers, agex_low, agex_high, months_from, months_to,
                            filterSTR=filter_str + [filterstr_ix],
                            variablesubset=['Vital: Wt-latest'],
                            modelType=modeltype_ix,
                            return_data_for_error_analysis=False,
                            return_data=True,
                            return_data_transformed=True,
                            return_train_test_data=True,
                            do_impute=False)

#             np.save(newdir+'/train_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: Wt-latest',str(months_to),'months.npy']), np.hstack((xtrain,mrnstrain.reshape(-1,1),ytrain.reshape(-1,1),ytrainlabel.reshape(-1,1))))
#             np.save(newdir+'/test_data_comb_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: Wt-latest',str(months_to),'months.npy']), np.hstack((xtest,mrnstest.reshape(-1,1),ytest.reshape(-1,1),ytestlabel.reshape(-1,1))))
#             np.save(newdir+'/data_transformed_'+'_'.join([modeltype_ix,gender_clean,mother_hist_ix,'Vital: Wt-latest',str(months_to)+'months.npy']), np.hstack((x2,mrns2.reshape(-1,1),y2.reshape(-1,1),y2label.reshape(-1,1))))
#             pickle.dump(feature_headers, open(newdir+'/feature_headers_pred_at_'+'_'.join([modeltype_ix,'Vital: Wt-latest',gender_clean,mother_hist_ix,str(months_to)+'months_transformed.pkl']), 'wb'))

            titles_total.append(gender_clean + ' - model: ' + modeltype_ix + ' - ' + 'Vital: Wt-latest @ ' + str(months_to) + 'months')
            model_list.append(model)
            prec_total.append(prec_list)
            recall_total.append(recall_list)
            spec_total.append(spec_list)
            auc_ste_list.append([test_auc_mean, test_auc_mean_ste])
            r2_list.append([r2test_mean, r2test_ste])


# Save the outputs
# pickle.dump(titles_total, open(newdir+'/titles_total.pkl', 'wb'))
# pickle.dump(model_list, open(newdir+'/model_list.pkl', 'wb'))
# pickle.dump(prec_total, open(newdir+'/prec_total.pkl', 'wb'))
# pickle.dump(recall_total, open(newdir+'/recall_total.pkl', 'wb'))
# pickle.dump(spec_total, open(newdir+'/spec_total.pkl', 'wb'))
# pickle.dump(auc_list, open(newdir+'/auc_list.pkl', 'wb'))
# pickle.dump(r2_list, open(newdir+'/r2_list.pkl', 'wb'))

/Volumes/CPO/obPY/src/zscore.py:57: RuntimeWarning: divide by zero encountered in true_divide
  zscores = (((weight / M)**L) - 1.) / (S * L)
/Volumes/CPO/obPY/src/zscore.py:57: RuntimeWarning: invalid value encountered in true_divide
  zscores = (((weight / M)**L) - 1.) / (S * L)
/Volumes/CPO/obPY/src/zscore.py:58: RuntimeWarning: invalid value encountered in greater
  zscores[(np.abs(zscores) > 5)] = np.sign(zscores[(np.abs(zscores) > 5)])
/Volumes/CPO/obPY/src/zscore.py:57: RuntimeWarning: divide by zero encountered in power
  zscores = (((weight / M)**L) - 1.) / (S * L)



Predicting obesity at 5 years from 6 months

lasso	Gender:0	no_maternal@ 6 months
Using pre-prepared data
5094 features are binary
x shape: (52945, 18912) num features: 18912
1380 patients selected..


/Volumes/CPO/obPY/src/train.py:168: RuntimeWarning: Mean of empty slice
  mu = np.nanmean(xcop, axis=0)
/Users/hammor01/anaconda3/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1427: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/Users/hammor01/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3167: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/hammor01/anaconda3/lib/python3.6/site-packages/numpy/lib/function_base.py:3168: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


18471
corr matrix is filtered to size (18912, 18471)
output is: average:16.909  min: 11.26  max: 38.514
normalizing output.
Predicting BMI at age:4.5 to 5.5years, from data in ages:0-6 months
filtering patients with:  ['Vital: Wt-avg0to1', 'Vital: Wt-avg1to3', 'Vital: BMI-avg0to1', 'Vital: BMI-avg1to3', 'Gender:0']
total size 1380
CV R^2 for alpha: 0.001 is: -0.143067134866
CV R^2 for alpha: 0.005 is: 0.0628495485704
CV R^2 for alpha: 0.01 is: 0.11616133507
CV R^2 for alpha: 0.1 is: 0.0921700384065
best alpha via CV: 0.01
AUC train: 0.711 Explained Variance Score Train: 0.180
AUC test: 0.679 Explained Variance Score Test: 0.116
CV R^2 for alpha: 0.001 is: -0.1096228005
CV R^2 for alpha: 0.005 is: -0.0207657684328
CV R^2 for alpha: 0.01 is: 0.0140718219478
CV R^2 for alpha: 0.1 is: 0.0357989521936
best alpha via CV: 0.1
AUC train: 0.664 Explained Variance Score Train: 0.084
AUC test: 0.579 Explained Variance Score Test: 0.036
CV R^2 for alpha: 0.001 is: -0.098141089118
CV R^2 for alpha:

* Vital: Wt-avg3to5 | coef 0.069 95% CI: [0.040 , 0.097] | OR_adj 1.071 [1.041 1.102] | occ: 1239 | OR_unadj: 2.078 [1.603 2.694] | indivs AUC:0.595
Correlated most with:
Newborn Diagnosis:9ccs219:Low birth wt:-0.186
    Maternal Diagnosis:9ccs185:Long pregncy:0.177
    Maternal Diagnosis:9ccs184:Early labor:-0.175
    Diagnosis:9ccs219:Low birth wt:-0.146
    Newborn Diagnosis:9ccs10:Immuniz/scrn:0.131
    Diagnosis:9ccs138:Esophgeal dx:-0.101
    Maternal Diagnosis:9ccs196:Nml preg/del:0.097
    Newborn Diagnosis:9ccs253:Allergy:-0.094
    Newborn Diagnosis:9ccs94:Other ear dx:0.092
    Newborn Diagnosis:9ccs218:Liveborn:0.086

* Newborn Diagnosis:9ccs222:Perint jaund | coef 0.058 95% CI: [0.012 , 0.104] | OR_adj 1.060 [1.012 1.110] | occ: 145 | OR_unadj: 1.500 [1.019 2.208] | indivs AUC:0.498
Correlated most with:
Newborn Diagnosis:9ccs222:Perint jaund:1.000
    Newborn Diagnosis:9ccs219:Low birth wt:0.335
    Maternal Diagnosis:9ccs184:Early labor:0.274
    Diagnosis:9ccs219:Low bi

* Maternal Diagnosis:9ccs195:Ot compl bir | coef 0.023 95% CI: [0.003 , 0.042] | OR_adj 1.023 [1.003 1.042] | occ: 590 | OR_unadj: 1.311 [1.013 1.695] | indivs AUC:0.496
Correlated most with:
Maternal Diagnosis:9ccs195:Ot compl bir:1.000
    Newborn Diagnosis:9ccs218:Liveborn:0.211
    Maternal Diagnosis:9ccs196:Nml preg/del:0.205
    Maternal Diagnosis:9ccs190:Fetal distrs:0.161
    MatDeliveryAge:0.140
    Newborn Diagnosis:9ccs94:Other ear dx:0.135
    Maternal Diagnosis:9ccs191:Amnios dx:0.130
    Newborn Diagnosis:9ccs10:Immuniz/scrn:0.126
    Newborn Diagnosis:9ccs224:Ot perint dx:0.122
    Maternal Diagnosis:9ccs185:Long pregncy:0.109

  Vital: Wt-gain3to7 | coef 0.022 95% CI: [0.001 , 0.043] | OR_adj 1.022 [1.001 1.044] | occ: 1286 | OR_unadj: 1.161 [0.889 1.518] | indivs AUC:0.491
Correlated most with:
Diagnosis:9ccs92:Otitis media:0.181
    Diagnosis:9ccs126:Ot up rsp in:0.146
    Diagnosis:9ccs125:Bronchitis:0.123
    Census:Percent; Estimate; Population 18 to 24 years - Bac

* Vital: Wt for Length Zscore-avg5to7 | coef 0.005 95% CI: [-0.004 , 0.015] | OR_adj 1.005 [0.996 1.015] | occ: 359 | OR_unadj: 1.575 [1.109 2.239] | indivs AUC:0.572
Correlated most with:
Maternal Diagnosis:9ccs55:Fluid/elc dx:-0.116
    Maternal Diagnosis:9ccs140:Gastritis:0.102
    Maternal Diagnosis:9ccs208:Acq foot def:-0.095
    Lab:292405 17-HYDROXYPROGESTERONE - Sunrise:0.091
    Diagnosis:9ccs224:Ot perint dx:-0.088
    Newborn Diagnosis:9ccs138:Esophgeal dx:-0.079
    Maternal Diagnosis:9ccs138:Esophgeal dx:-0.076
    Maternal Diagnosis:9ccs107:Cardia arrst:-0.075
    Diagnosis:9ccs222:Perint jaund:-0.072
    Diagnosis:9ccs258:Other screen:0.071

  Diagnosis:9ccs126:Ot up rsp in | coef -0.005 95% CI: [-0.014 , 0.004] | OR_adj 0.995 [0.986 1.004] | occ: 552 | OR_unadj: 0.802 [0.615 1.046] | indivs AUC:0.498
Correlated most with:
Diagnosis:9ccs126:Ot up rsp in:1.000
    Diagnosis:9ccs125:Bronchitis:0.156
    Diagnosis:9ccs92:Otitis media:0.139
    Diagnosis:9ccs133:Oth low resp

Maternal Diagnosis : 0.32218939299
Vital : 0.299548017272
Newborn Diagnosis : 0.047614108358
Lab : 0.0259691155915
Census : 0.000777555345829
Diagnosis : -0.00182927488559
MatDeliveryAge : 0.00477417519454
lasso	Gender:0	no_maternal@ 6 months w/o vitals and maternal historical data
Using pre-prepared data
5094 features are binary
x shape: (52945, 18912) num features: 18912
1687 patients selected..
18471
corr matrix is filtered to size (18912, 18471)
subsetting variables that are only: ['Census', 'Diagnosis', 'Ethnicity', 'Gender', 'Lab', 'MatDeliveryAge', 'Medication', 'Newborn Diagnosis', 'Prim_Insur', 'Race', 'Second_Insur', 'Zipcode']
from  18912  variables to  5547
output is: average:16.871  min: 11.26  max: 38.514
normalizing output.
Predicting BMI at age:4.5 to 5.5years, from data in ages:0-6 months
filtering patients with:  ['Gender:0']
total size 1687
CV R^2 for alpha: 0.001 is: -0.134517596917
CV R^2 for alpha: 0.005 is: -0.0201346569589
CV R^2 for alpha: 0.01 is: -0.004244956

IndexError: boolean index did not match indexed array along dimension 1; dimension is 18912 but corresponding boolean dimension is 5547

In [6]:
x1.shape

(52945, 18912)

In [7]:
bin_ix = np.array([(h.startswith('Diagnosis:') or h.startswith('Maternal Diagnosis:') or h.startswith('Newborn Diagnosis:')) for h in feature_headers])

In [8]:
bin_ix.shape

(5547,)